<a href="https://colab.research.google.com/github/kuds/rl-unity-soccer/blob/main/%5BSoccerTwo%5D%20Training%20Agent%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://download.unity3d.com/download_unity/0e25a174756c/UnitySetup-2023.2.20f1" -O UnitySetup
!chmod +x UnitySetup

--2024-09-03 01:47:35--  https://download.unity3d.com/download_unity/0e25a174756c/UnitySetup-2023.2.20f1
Resolving download.unity3d.com (download.unity3d.com)... 23.46.216.85, 23.46.216.71
Connecting to download.unity3d.com (download.unity3d.com)|23.46.216.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 762560 (745K) [application/octet-stream]
Saving to: ‘UnitySetup’

UnitySetup          100%[===================>] 744.69K  --.-KB/s    in 0.1s    

2024-09-03 01:47:35 (5.39 MB/s) - ‘UnitySetup’ saved [762560/762560]



In [12]:
!./UnitySetup --help

Usage:
  UnitySetup [OPTION…]

Help Options:
  -h, --help                           Show help options

Application Options:
  -i, --inifile=FILE                   Use local ini file FILE
  -u, --unattended                     Perform unattended install
  -L, --list-components                List available components to install
  -l, --install-location=LOCATION      Use install location LOCATION in unattended mode
  -d, --download-location=LOCATION     Use download location LOCATION in unattended mode
  -c, --components=COMPONENTS          Install comma separated list of components
  --save-component-list=FILE           Save component list to FILE
  --use-component-list=FILE            Use component list from FILE
  -v, --verbose                        Print verbose output



In [13]:
!yes Y | ./UnitySetup --unattended --components="Unity" --install-location="/opt" --no-graphics

./UnitySetup: Unknown option --no-graphics


In [1]:
# Clone the repository
!git clone https://github.com/Unity-Technologies/ml-agents

fatal: destination path 'ml-agents' already exists and is not an empty directory.


In [2]:
# Go inside the repository and install the package
!pip3 install ./ml-agents/ml-agents-envs

Processing ./ml-agents/ml-agents-envs
  Preparing metadata (setup.py) ... done
  Created wheel for mlagents_envs: filename=mlagents_envs-1.1.0.dev0-py3-none-any.whl size=107881 sha256=b221d7818c90529364110115701f86ca9372087b828aefb9163b1bc5fe9f546d
  Stored in directory: /root/.cache/pip/wheels/06/a4/44/559ecb18c6173396ce39acd3f9593bfb562b00fa707a216311
Successfully built mlagents_envs
  Attempting uninstall: mlagents_envs
    Found existing installation: mlagents_envs 1.1.0.dev0
    Uninstalling mlagents_envs-1.1.0.dev0:
      Successfully uninstalled mlagents_envs-1.1.0.dev0


In [3]:
# Go inside the repository and install the package
!pip3 install ./ml-agents/ml-agents

Processing ./ml-agents/ml-agents
  Preparing metadata (setup.py) ... done
  Created wheel for mlagents: filename=mlagents-1.1.0.dev0-py3-none-any.whl size=171442 sha256=7587ab4f1586eb2a322cbf50bd27332c43a0bea9f119e004430291ca10fb392a
  Stored in directory: /root/.cache/pip/wheels/90/f9/f3/05444ae1e763d3ed872e12f2ca47ef57ad9f8045d927bf68ad
Successfully built mlagents
  Attempting uninstall: mlagents
    Found existing installation: mlagents 1.1.0.dev0
    Uninstalling mlagents-1.1.0.dev0:
      Successfully uninstalled mlagents-1.1.0.dev0


In [4]:
import platform
import mlagents
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.envs import SoccerTwos
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.registry import default_registry
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import matplotlib.pyplot as plt

from importlib.metadata import version

In [5]:
print("Python version:", platform.python_version())
print("PyTorch version:", version("torch"))
print("ML-Agents version:", version("mlagents"))
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0)) if torch.cuda.is_available() else print("No GPU available")

Python version: 3.10.12
PyTorch version: 2.4.0+cu121
ML-Agents version: 1.1.0.dev0
CUDA available: False
No GPU available


In [6]:
class DQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQNetwork(state_size, action_size)
        self.target_model = DQNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.update_target_model()

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
        act_values = self.model(state)
        return np.argmax(act_values.detach().numpy()[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state = torch.tensor(next_state, dtype=torch.float).unsqueeze(0)
                target = reward + self.gamma * np.amax(self.target_model(next_state).detach().numpy())
            state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            target_f = self.model(state).detach().numpy()
            target_f[0][action] = target
            target_f = torch.tensor(target_f)
            self.optimizer.zero_grad()
            outputs = self.model(state)
            loss = nn.MSELoss()(outputs, target_f)
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [19]:
from mlagents_envs.registry import default_registry

env = default_registry["SoccerTwos"].make(no_graphics=True)
#env.reset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlagents_envs/environment.py:94: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  unity_communicator_version = StrictVersion(unity_com_ver)


In [8]:
env_path = "opt/Editor/env"
engine_channel = EngineConfigurationChannel()
env = UnityEnvironment(file_name=env_path, side_channels=[engine_channel])
engine_channel.set_configuration_parameters(time_scale=20.0)
env.reset()

UnityEnvironmentException: Couldn't launch the opt/Editor/env environment. Provided filename does not match any environments.

In [15]:
!mlagents-learn --no-graphics

2024-09-03 02:14:48.075050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 02:14:48.099630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 02:14:48.106683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 02:14:48.124556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 02:14:49.541438: W tensorflow/compiler/tf2